In [ ]:
import os
print("Estoy trabajando en:", os.getcwd())
print("Contenido de la carpeta actual:")
print(os.listdir())


Estoy trabajando en: c:\Users\braya\Documents\E-COMMERCE RESEARCH\aurex-demand-forecasting\notebooks
Contenido de la carpeta actual:
['01_eda.ipynb', '02_baseline_arima.ipynb', '03_sarima_baseline.ipynb', '04_lstm_attention.ipynb', '05_hybrid_arima_lstm.ipynb', '06_dae_lstm.ipynb', '07_lstm_stacked.ipynb', '08_lstm_bidirectional.ipynb', '09_seq2seq_lstm.ipynb', '10_tcn_baseline.ipynb', '11_transformer_tft.ipynb', '12_m5_dataset_build.ipynb', '13_sarimax_multivariate_m5.ipynb', '14_lstm_attention_m5.ipynb', '15_hybrid_arima_lstm_m5.ipynb', '16_dae_lstm_m5.ipynb', '17_transformer_m5.ipynb', 'data']


In [ ]:
import os

# Buscar archivos CSV en TODAS las subcarpetas
print("🔍 BUSCANDO ARCHIVOS CSV...")
print(f"Buscando desde: {os.getcwd()}\n")

encontrados = 0
for root, dirs, files in os.walk("."):
    for file in files:
        if file.endswith('.csv') and 'calendar' in file:
            encontrados += 1
            print(f"✅ Encontrado: {os.path.join(root, file)}")
        elif file.endswith('.csv') and 'sell_prices' in file:
            encontrados += 1
            print(f"✅ Encontrado: {os.path.join(root, file)}")
        elif file.endswith('.csv') and 'sales_train' in file:
            encontrados += 1
            print(f"✅ Encontrado: {os.path.join(root, file)}")

if encontrados == 0:
    print("❌ No se encontraron archivos CSV")
else:
    print(f"\n🎯 Total archivos encontrados: {encontrados}")

🔍 BUSCANDO ARCHIVOS CSV...
Buscando desde: c:\Users\braya\Documents\E-COMMERCE RESEARCH\aurex-demand-forecasting\notebooks

❌ No se encontraron archivos CSV


In [ ]:
import pandas as pd
import os

def encontrar_y_cargar_m5():
    """Busca automáticamente los archivos del M5 y los carga"""
    
    print("🔍 Buscando archivos del dataset M5...\n")
    
    # Posibles ubicaciones
    posibles_rutas = [
        ".",  # carpeta actual
        "data",
        "../data",
        "raw",
        "data/raw",
        "../data/raw",
        "notebooks/data",
        "../notebooks/data"
    ]
    
    archivos_necesarios = {
        'calendar': 'calendar.csv',
        'prices': 'sell_prices.csv',
        'sales': 'sales_train_validation.csv'
    }
    
    encontrados = {}
    
    # Buscar cada archivo
    for nombre, archivo in archivos_necesarios.items():
        for ruta in posibles_rutas:
            ruta_completa = os.path.join(ruta, archivo)
            if os.path.exists(ruta_completa):
                encontrados[nombre] = ruta_completa
                print(f"✅ {archivo} encontrado en: {ruta_completa}")
                break
    
    print("\n" + "="*50)
    
    # Cargar archivos encontrados
    if len(encontrados) == 3:
        print("\n📂 CARGANDO ARCHIVOS...")
        
        calendar = pd.read_csv(encontrados['calendar'])
        print(f"✅ Calendar cargado: {calendar.shape}")
        
        prices = pd.read_csv(encontrados['prices'])
        print(f"✅ Prices cargado: {prices.shape}")
        
        sales = pd.read_csv(encontrados['sales'])
        print(f"✅ Sales cargado: {sales.shape}")
        
        return calendar, prices, sales
    else:
        print("\n❌ No se encontraron todos los archivos necesarios")
        print(f"Archivos encontrados: {list(encontrados.keys())}")
        return None, None, None

# EJECUTAR
calendar, prices, sales = encontrar_y_cargar_m5()

🔍 Buscando archivos del dataset M5...

✅ calendar.csv encontrado en: ../data/raw\calendar.csv
✅ sell_prices.csv encontrado en: ../data/raw\sell_prices.csv
✅ sales_train_validation.csv encontrado en: ../data/raw\sales_train_validation.csv


📂 CARGANDO ARCHIVOS...
✅ Calendar cargado: (1969, 14)
✅ Prices cargado: (6841121, 4)
✅ Sales cargado: (30490, 1919)


In [ ]:
# ============================================
# NOTEBOOK 12: m5_dataset_build.ipynb
# VERSIÓN MEJORADA (v2) - +REALISTA +ROBUSTA +SPLITS
# ============================================

import os
import numpy as np
import pandas as pd

# ------------------------------------------------------------
# 0) CONFIG
# ------------------------------------------------------------
RAW_DIR = "../data/raw"
OUT_DIR = "../data/processed"
os.makedirs(OUT_DIR, exist_ok=True)

N_DAYS = 500      # últimos N días (más realista)
TOP_N = 3         # top productos por ventas en ventana N_DAYS

print(f"📁 Directorio actual: {os.getcwd()}")
print(f"📂 RAW_DIR: {RAW_DIR}")
print(f"📂 OUT_DIR: {OUT_DIR}")
print(f"🧩 N_DAYS (últimos): {N_DAYS}")
print(f"🏆 TOP_N: {TOP_N}\n")

CAL_PATH = os.path.join(RAW_DIR, "calendar.csv")
PRICE_PATH = os.path.join(RAW_DIR, "sell_prices.csv")
SALES_PATH = os.path.join(RAW_DIR, "sales_train_validation.csv")

# ------------------------------------------------------------
# 1) CARGAR DATOS (OPTIMIZADO)
# ------------------------------------------------------------
print("📊 Cargando archivos...")

calendar = pd.read_csv(CAL_PATH)
print(f"  ✅ calendar.csv: {calendar.shape}")

sales = pd.read_csv(
    SALES_PATH,
    dtype={
        "id": "category",
        "item_id": "category",
        "dept_id": "category",
        "cat_id": "category",
        "store_id": "category",
        "state_id": "category",
    },
)
print(f"  ✅ sales_train_validation.csv: {sales.shape}")

prices = pd.read_csv(
    PRICE_PATH,
    dtype={
        "item_id": "category",
        "store_id": "category",
        "wm_yr_wk": "int32",
        "sell_price": "float32",
    },
)
print(f"  ✅ sell_prices.csv: {prices.shape}")

# ------------------------------------------------------------
# 2) WIDE → LONG (ÚLTIMOS N_DAYS)
# ------------------------------------------------------------
print("\n🔄 Transformando sales de wide a long...")

id_cols = ["id", "item_id", "dept_id", "cat_id", "store_id", "state_id"]
d_cols = [c for c in sales.columns if c.startswith("d_")]

# ✅ Más realista: últimos N días (no los primeros)
d_cols = d_cols[-N_DAYS:]

sales_long = sales[id_cols + d_cols].melt(
    id_vars=id_cols,
    value_vars=d_cols,
    var_name="d",
    value_name="sales",
)

# liberar memoria
del sales

print(f"  ✅ sales_long: {sales_long.shape}")
print(f"  📊 Memoria aprox: {sales_long.memory_usage(deep=True).sum() / 1024**2:.1f} MB")

# ------------------------------------------------------------
# 3) MERGE con CALENDAR
# ------------------------------------------------------------
print("\n🔄 Merge con calendar...")

calendar["date"] = pd.to_datetime(calendar["date"])
calendar["d"] = calendar["d"].astype("category")

cal_keep = [
    "d", "date", "wm_yr_wk", "weekday", "wday", "month", "year",
    "event_name_1", "event_type_1", "event_name_2", "event_type_2",
    "snap_CA", "snap_TX", "snap_WI",
]

calendar_small = calendar[cal_keep].copy()
calendar_small["wm_yr_wk"] = calendar_small["wm_yr_wk"].astype("int32")

df = sales_long.merge(calendar_small, on="d", how="left")
del sales_long

print(f"  ✅ Merged with calendar: {df.shape}")

# ------------------------------------------------------------
# 4) MERGE con PRICES
# ------------------------------------------------------------
print("\n🔄 Merge con precios...")

df = df.merge(prices, on=["store_id", "item_id", "wm_yr_wk"], how="left")
print(f"  ✅ Merged with prices: {df.shape}")

# ------------------------------------------------------------
# 5) LIMPIEZA ROBUSTA (precios + sales)
# ------------------------------------------------------------
print("\n🧹 Limpiando datos...")

df = df.sort_values(["id", "date"]).reset_index(drop=True)

# Sales
df["sales"] = df["sales"].fillna(0).astype("float32")

# ✅ Precio robusto:
# 1) forward fill por SKU-store (id)
df["sell_price"] = df.groupby("id", observed=True)["sell_price"].ffill()

# 2) fallback: mediana por item_id
item_median = df.groupby("item_id", observed=True)["sell_price"].median()
global_median = df["sell_price"].median()

df["sell_price"] = df["sell_price"].fillna(df["item_id"].map(item_median))
df["sell_price"] = df["sell_price"].fillna(global_median)

# sanity
print("  ✅ sell_price NaNs:", int(df["sell_price"].isna().sum()))
print("  ✅ sales NaNs:", int(df["sales"].isna().sum()))

# ------------------------------------------------------------
# 6) FEATURES CALENDARIO / EVENTOS / SNAP
# ------------------------------------------------------------
print("\n🔧 Creando features básicas...")

df["dayofweek"] = df["date"].dt.dayofweek.astype("int8")
df["is_weekend"] = (df["dayofweek"] >= 5).astype("int8")
df["has_event_1"] = df["event_name_1"].notna().astype("int8")
df["has_event_2"] = df["event_name_2"].notna().astype("int8")

# SNAP por estado (unificar en 1 feature)
df["snap"] = 0
for state in ["CA", "TX", "WI"]:
    mask = (df["state_id"] == state)
    df.loc[mask, "snap"] = df.loc[mask, f"snap_{state}"].astype("int8")

df["snap"] = df["snap"].astype("int8")

# ------------------------------------------------------------
# 7) TOP-N SKUs por ventas (en esta ventana N_DAYS)
# ------------------------------------------------------------
print(f"\n🏆 Seleccionando top {TOP_N} productos...")

sku_rank = df.groupby("id", observed=True)["sales"].sum().sort_values(ascending=False)
top_ids = sku_rank.head(TOP_N).index.tolist()

print(f"  ✅ Top {TOP_N} IDs:")
for i, pid in enumerate(top_ids, 1):
    print(f"     {i}. {pid}: {float(sku_rank[pid]):,.0f} ventas")

df_top = df[df["id"].isin(top_ids)].copy()
df_top = df_top.sort_values(["id", "date"]).reset_index(drop=True)

print(f"  ✅ df_top (antes lags): {df_top.shape}")

# liberar df grande
del df

# ------------------------------------------------------------
# 8) LAGS + ROLLING (por SKU)
# ------------------------------------------------------------
print("\n⏳ Creando lags para top productos...")

df_top["lag_7"]  = df_top.groupby("id", observed=True)["sales"].shift(7)
df_top["lag_14"] = df_top.groupby("id", observed=True)["sales"].shift(14)
df_top["lag_28"] = df_top.groupby("id", observed=True)["sales"].shift(28)

df_top["roll_mean_7"] = (
    df_top.groupby("id", observed=True)["sales"]
          .shift(1)
          .rolling(7, min_periods=7)
          .mean()
          .reset_index(level=0, drop=True)
)

df_top["roll_mean_14"] = (
    df_top.groupby("id", observed=True)["sales"]
          .shift(1)
          .rolling(14, min_periods=14)
          .mean()
          .reset_index(level=0, drop=True)
)

print(f"  ✅ df_top con lags: {df_top.shape}")

# ------------------------------------------------------------
# 9) DROP NA (solo lo crítico)
# ------------------------------------------------------------
print("\n🧹 Eliminando filas con NaN...")

initial_rows = len(df_top)

# ✅ Eliminamos donde lags/rolling no existen aún (inicio de serie)
df_top = df_top.dropna(
    subset=["lag_7", "lag_14", "lag_28", "roll_mean_7", "roll_mean_14"]
).copy()

dropped_rows = initial_rows - len(df_top)

print(f"  ✅ Filas iniciales: {initial_rows}")
print(f"  ✅ Filas después de dropna: {len(df_top)}")
print(f"  ✅ Filas eliminadas: {dropped_rows} ({(dropped_rows/initial_rows)*100:.1f}%)")

# ------------------------------------------------------------
# 10) DATASET FINAL + SPLITS (train/val/test por SKU)
# ------------------------------------------------------------
print("\n🧩 Construyendo dataset final + splits...")

columns_to_save = [
    "id", "date", "sales",
    "sell_price", "snap",
    "dayofweek", "is_weekend",
    "has_event_1", "has_event_2",
    "lag_7", "lag_14", "lag_28",
    "roll_mean_7", "roll_mean_14",
]

df_top_ml = df_top[columns_to_save].copy()
df_top_ml = df_top_ml.sort_values(["id", "date"]).reset_index(drop=True)

def add_split(g):
    n = len(g)
    g = g.copy()
    g["split"] = "train"
    g.loc[int(0.8*n):int(0.9*n), "split"] = "val"
    g.loc[int(0.9*n):, "split"] = "test"
    return g

df_top_ml = df_top_ml.groupby("id", group_keys=False).apply(add_split)

print("  ✅ Split counts:\n", df_top_ml["split"].value_counts())

# ------------------------------------------------------------
# 11) GUARDAR (CSV + sample)
# ------------------------------------------------------------
print("\n💾 Guardando dataset procesado...")

out_csv = os.path.join(OUT_DIR, "m5_top3_multivariate.csv")
out_sample = os.path.join(OUT_DIR, "m5_top3_sample.csv")

df_top_ml.to_csv(out_csv, index=False)
df_top_ml.head(1000).to_csv(out_sample, index=False)

print("\n✅ Archivos guardados exitosamente:")
print(f"  - {out_csv}")
print(f"  - {out_sample}")

print("\n📊 Dataset final:")
print(f"  - Productos: {df_top_ml['id'].nunique()}")
print(f"  - Fechas: {df_top_ml['date'].min()} → {df_top_ml['date'].max()}")
print(f"  - Registros: {len(df_top_ml):,}")
print(f"  - Columnas: {df_top_ml.columns.tolist()}")

print("\n🎯 ¡PROCESAMIENTO COMPLETADO (v2)!")


📁 Directorio actual: c:\Users\braya\Documents\E-COMMERCE RESEARCH\aurex-demand-forecasting\notebooks
📂 RAW_DIR: ../data/raw

📊 Cargando archivos...
  ✅ calendar.csv: (1969, 14)
  ✅ sales_train_validation.csv: (30490, 1919)
  ✅ sell_prices.csv: (6841121, 4)

🔄 Transformando sales de wide a long...
  ✅ sales_long: (15245000, 8)
  📊 Memoria: 1134.7 MB

🔄 Merge con calendar...
  ✅ Merged with calendar: (15245000, 21)

🔄 Merge con precios...
  ✅ Merged with prices: (15245000, 22)

🧹 Limpiando datos...


c:\Users\braya\Documents\E-COMMERCE RESEARCH\aurex-demand-forecasting\.venv\Lib\site-packages\numpy\lib\_nanfunctions_impl.py:1213: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
c:\Users\braya\Documents\E-COMMERCE RESEARCH\aurex-demand-forecasting\.venv\Lib\site-packages\numpy\lib\_nanfunctions_impl.py:1213: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
c:\Users\braya\Documents\E-COMMERCE RESEARCH\aurex-demand-forecasting\.venv\Lib\site-packages\numpy\lib\_nanfunctions_impl.py:1213: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
c:\Users\braya\Documents\E-COMMERCE RESEARCH\aurex-demand-forecasting\.venv\Lib\site-packages\numpy\lib\_nanfunctions_impl.py:1213: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
c:\Users\braya\Documents\E-COMMERCE RESEARCH\aurex-demand-forecasting\.venv\Lib\site-packages\numpy\

🔧 Creando features básicas...

🏆 Seleccionando top 3 productos...
  ✅ Top 3 IDs:
     1. FOODS_3_090_CA_3_validation: 58,437 ventas
     2. FOODS_3_586_TX_2_validation: 54,097 ventas
     3. FOODS_3_586_TX_3_validation: 47,007 ventas
  ✅ df_top: (1500, 27)

⏳ Creando lags para top productos...
  ✅ Lags creados
  📊 df_top con lags: (1500, 32)

🧹 Eliminando filas con NaN...
  ✅ Filas iniciales: 1500
  ✅ Filas después de dropna: 1458
  ✅ Filas eliminadas: 42 (2.8%)

💾 Guardando dataset procesado...

✅ Archivos guardados exitosamente:
  - ../data/processed\m5_top3_multivariate.csv
  - ../data/processed\m5_top3_sample.csv

📊 Dataset final:
  - Productos: 3
  - Fechas: 2011-02-12 00:00:00 → 2012-06-11 00:00:00
  - Registros: 1,458
  - Columnas: ['id', 'date', 'sales', 'sell_price', 'snap', 'dayofweek', 'is_weekend', 'has_event_1', 'has_event_2', 'lag_7', 'lag_14', 'lag_28', 'roll_mean_7', 'roll_mean_14']

🎯 ¡PROCESAMIENTO COMPLETADO!
